In [2]:
#CONNECT TO SPOTIFY
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="874ee9fbf333484c9525c641ad56b51e" #get these from spotify for developers website
secret = "d62f64e50b084f1e8b380581c5b7993c"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [15]:
#GET SONG NAME FROM PLAYLIST


# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
ids = [] 

#Get playlist
track_results = sp.user_playlist("Pei-Hsuan Lin", "37i9dQZEVXbMDoHDwVN2tF")["tracks"]["items"] #edit this to put in your spotify playlist URI code


for t in track_results:
    artist_name.append(t['track']['artists'][0]['name'])
    track_name.append(t['track']['name'])
    track_id.append(t['track']['id'])
    popularity.append(t['track']['popularity'])

print('number of elements in the track_id list:', len(track_id))

#imports track names
import pandas as pd
dftracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(dftracks.shape)

dftracks.shape
dftracks.info()
dftracks.head()


features = sp.audio_features(track_id) 
dfmet = pd.DataFrame(features)
dfmet.rename(columns={'id': 'track_id'}, inplace=True)
dfmet.info()

number of elements in the track_id list: 50
(50, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_name  50 non-null     object
 1   track_name   50 non-null     object
 2   track_id     50 non-null     object
 3   popularity   50 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      50 non-null     float64
 1   energy            50 non-null     float64
 2   key               50 non-null     int64  
 3   loudness          50 non-null     float64
 4   mode              50 non-null     int64  
 5   speechiness       50 non-null     float64
 6   acousticness      50 non-null     float64
 7   instrumentalness  5

In [6]:
#SONG NAME DATA CLEANUP

grouped = dftracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

dftracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

# doing the same grouping as before to verify the solution
grouped_after_dropping = dftracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

dftracks[dftracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()
dftracks.shape

(50, 4)

In [16]:
# MERGE BOTH DATAFRAMES

# the 'inner' method will make sure to keep track IDs present in both datasets
dftotal = pd.merge(dftracks,dfmet,on='track_id',how='inner')
dftotal.info()

dftotal[dftotal.duplicated(subset=['artist_name','track_name'],keep=False)]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_name       50 non-null     object 
 1   track_name        50 non-null     object 
 2   track_id          50 non-null     object 
 3   popularity        50 non-null     int64  
 4   danceability      50 non-null     float64
 5   energy            50 non-null     float64
 6   key               50 non-null     int64  
 7   loudness          50 non-null     float64
 8   mode              50 non-null     int64  
 9   speechiness       50 non-null     float64
 10  acousticness      50 non-null     float64
 11  instrumentalness  50 non-null     float64
 12  liveness          50 non-null     float64
 13  valence           50 non-null     float64
 14  tempo             50 non-null     float64
 15  type              50 non-null     object 
 16  uri               50 non-null     object 
 17 

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature


In [17]:
dftotal.head()

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Drake,Toosie Slide,466cKvZn1j45IpxDdYZqdA,0,0.830,0.490,1,-8.820,0,0.2090,...,0.000003,0.1130,0.845,81.604,audio_features,spotify:track:466cKvZn1j45IpxDdYZqdA,https://api.spotify.com/v1/tracks/466cKvZn1j45...,https://api.spotify.com/v1/audio-analysis/466c...,247059,4
1,The Weeknd,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,100,0.514,0.730,1,-5.934,1,0.0598,...,0.000095,0.0897,0.334,171.005,audio_features,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4
2,THE SCOTTS,THE SCOTTS,39Yp9wwQiSRIDOvrVg7mbk,94,0.716,0.537,0,-7.648,0,0.0514,...,0.000000,0.1570,0.280,129.979,audio_features,spotify:track:39Yp9wwQiSRIDOvrVg7mbk,https://api.spotify.com/v1/tracks/39Yp9wwQiSRI...,https://api.spotify.com/v1/audio-analysis/39Yp...,165978,4
3,SAINt JHN,Roses - Imanbek Remix,24Yi9hE78yPEbZ4kxyoXAI,98,0.770,0.724,8,-5.484,1,0.0495,...,0.010700,0.3530,0.898,121.975,audio_features,spotify:track:24Yi9hE78yPEbZ4kxyoXAI,https://api.spotify.com/v1/tracks/24Yi9hE78yPE...,https://api.spotify.com/v1/audio-analysis/24Yi...,176840,4
4,Powfu,death bed (coffee for your head) (feat. beabad...,7eJMfftS33KTjuF7lTsMCx,99,0.726,0.431,8,-8.765,0,0.1350,...,0.000000,0.6960,0.348,144.026,audio_features,spotify:track:7eJMfftS33KTjuF7lTsMCx,https://api.spotify.com/v1/tracks/7eJMfftS33KT...,https://api.spotify.com/v1/audio-analysis/7eJM...,173333,4


In [18]:
#write to csv
dftotal.to_csv('SpotifyHits50.csv')

In [19]:
dftotal

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Drake,Toosie Slide,466cKvZn1j45IpxDdYZqdA,0,0.830,0.490,1,-8.820,0,0.2090,...,0.000003,0.1130,0.8450,81.604,audio_features,spotify:track:466cKvZn1j45IpxDdYZqdA,https://api.spotify.com/v1/tracks/466cKvZn1j45...,https://api.spotify.com/v1/audio-analysis/466c...,247059,4
1,The Weeknd,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,100,0.514,0.730,1,-5.934,1,0.0598,...,0.000095,0.0897,0.3340,171.005,audio_features,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4
2,THE SCOTTS,THE SCOTTS,39Yp9wwQiSRIDOvrVg7mbk,94,0.716,0.537,0,-7.648,0,0.0514,...,0.000000,0.1570,0.2800,129.979,audio_features,spotify:track:39Yp9wwQiSRIDOvrVg7mbk,https://api.spotify.com/v1/tracks/39Yp9wwQiSRI...,https://api.spotify.com/v1/audio-analysis/39Yp...,165978,4
3,SAINt JHN,Roses - Imanbek Remix,24Yi9hE78yPEbZ4kxyoXAI,98,0.770,0.724,8,-5.484,1,0.0495,...,0.010700,0.3530,0.8980,121.975,audio_features,spotify:track:24Yi9hE78yPEbZ4kxyoXAI,https://api.spotify.com/v1/tracks/24Yi9hE78yPE...,https://api.spotify.com/v1/audio-analysis/24Yi...,176840,4
4,Powfu,death bed (coffee for your head) (feat. beabad...,7eJMfftS33KTjuF7lTsMCx,99,0.726,0.431,8,-8.765,0,0.1350,...,0.000000,0.6960,0.3480,144.026,audio_features,spotify:track:7eJMfftS33KTjuF7lTsMCx,https://api.spotify.com/v1/tracks/7eJMfftS33KT...,https://api.spotify.com/v1/audio-analysis/7eJM...,173333,4
5,Dua Lipa,Don't Start Now,3PfIrDoz19wz7qK7tYeu62,91,0.794,0.793,11,-4.521,0,0.0842,...,0.000000,0.0952,0.6770,123.941,audio_features,spotify:track:3PfIrDoz19wz7qK7tYeu62,https://api.spotify.com/v1/tracks/3PfIrDoz19wz...,https://api.spotify.com/v1/audio-analysis/3PfI...,183290,4
6,Tones And I,Dance Monkey,1rgnBhdG2JDFTbYkYRZAku,75,0.825,0.593,6,-6.401,0,0.0988,...,0.000161,0.1700,0.5400,98.078,audio_features,spotify:track:1rgnBhdG2JDFTbYkYRZAku,https://api.spotify.com/v1/tracks/1rgnBhdG2JDF...,https://api.spotify.com/v1/audio-analysis/1rgn...,209755,4
7,Drake,Pain 1993 (with Playboi Carti),6Kj17Afjo1OKJYpf5VzCeo,0,0.825,0.373,5,-5.532,0,0.1080,...,0.000000,0.3680,0.1200,77.271,audio_features,spotify:track:6Kj17Afjo1OKJYpf5VzCeo,https://api.spotify.com/v1/tracks/6Kj17Afjo1OK...,https://api.spotify.com/v1/audio-analysis/6Kj1...,149078,4
8,Roddy Ricch,The Box,0nbXyq5TXYPCO7pr3N8S4I,99,0.896,0.586,10,-6.687,0,0.0559,...,0.000000,0.7900,0.6420,116.971,audio_features,spotify:track:0nbXyq5TXYPCO7pr3N8S4I,https://api.spotify.com/v1/tracks/0nbXyq5TXYPC...,https://api.spotify.com/v1/audio-analysis/0nbX...,196653,4
9,Doja Cat,Say So,3Dv1eDb0MEgF93GpLXlucZ,97,0.787,0.673,11,-4.577,0,0.1580,...,0.000004,0.0904,0.7860,110.962,audio_features,spotify:track:3Dv1eDb0MEgF93GpLXlucZ,https://api.spotify.com/v1/tracks/3Dv1eDb0MEgF...,https://api.spotify.com/v1/audio-analysis/3Dv1...,237893,4
